In [1]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '../core'))
import kaggle_support as kgs
import importlib
import matplotlib.pyplot as plt
import numpy as np
import cupy as cp
from dataclasses import dataclass, field, fields
import pack_cuda
import pack_vis
import pack_cost
import copy
import time
import pack_ga
from IPython.display import HTML, display, clear_output
kgs.set_float32(True)


vast
vast
stop final relax at some point


In [2]:
1/1.7143

0.5833284722627312

In [3]:
importlib.reload(pack_ga)
fastMode = False

# fast for now
#runner.population_size = 1
#runner.selection_size = [0]
#runner.n_generations = 500

# structural
ratios = np.unique(np.concatenate((np.arange(0.44,0.49,0.002), np.arange(0.56,0.6,0.002), np.arange(0.4,0.6001,0.01), np.arange(0.3,1.0001,0.02))))
print(len(ratios))
res = []
for r in ratios:
    print(r)
    runner = pack_ga.GA()
    runner.N_trees_to_do = np.array([2])
    runner.population_size = 10000
    runner.initializer.size_setup *= 10
    runner.plot_fitness_predictors = False
    runner.plot_diversity_matrix = False
    runner.plot_champions = False
    runner.initializer.base_solution = kgs.SolutionCollectionLatticeFixed()
    runner.initializer.base_solution.aspect_ratios = cp.array([1/r], dtype=kgs.dtype_cp)
    runner.initializer.jiggler.duration_init = 0.
    runner.initializer.jiggler.duration_compact /= 100000
    runner.initializer.jiggler.duration_final = 0.
    runner.initializer.jiggler.plot_interval = None
    runner.initializer.jiggler.n_rounds = 1
    runner.rough_relaxers[0].plot_interval =None
    runner.rough_relaxers[0].n_iterations//=5
    runner.fine_relaxers[0].plot_interval=None
    runner.rough_relaxers[0].cost.costs.pop(1)
    runner.fine_relaxers[0].cost.costs.pop(1)
    runner.fine_relaxers[0].n_iterations//=1
    runner.fine_relaxers[0].dt/=2
    runner.fine_relaxers.append(copy.deepcopy(runner.fine_relaxers[0]))
    runner.fine_relaxers[1].cost.costs[0].scaling/=2
    runner.fine_relaxers.append(copy.deepcopy(runner.fine_relaxers[0]))
    runner.fine_relaxers[1].cost.costs[0].scaling/=4
    runner.n_generations = 2
    
    runner.fitness_cost.costs.pop(1)
    kgs.debugging_mode = 1
    kgs.profiling=False
    new_moves = []
    for i in [1,2]:
        new_moves.append(runner.move.moves[i])
    runner.move.moves = new_moves
    #print(runner.move.moves)
    #runner.rough_relaxers = []
    #runner.fine_relaxers = []
    runner.run()
    res.append(runner)

stop final relax at some point
102
0.3
Generation 0, Trees 2, Best cost: 0.00950821, Est: 0.47541061, h: 0.529785
[[0.5297853]]
Generation 1, Trees 2, Best cost: 0.00950821, Est: 0.47541052, h: 0.529778
[[0.5297777]]
0.32
Generation 0, Trees 2, Best cost: 0.00892309, Est: 0.44615465, h: 0.530325
[[0.53032535]]
Generation 1, Trees 2, Best cost: 0.00892309, Est: 0.44615451, h: 0.530318
[[0.5303177]]
0.34
Generation 0, Trees 2, Best cost: 0.00840581, Est: 0.42029047, h: 0.530795
[[0.53079486]]
Generation 1, Trees 2, Best cost: 0.00840581, Est: 0.42029038, h: 0.530793
[[0.5307926]]
0.36000000000000004
Generation 0, Trees 2, Best cost: 0.00794522, Est: 0.39726081, h: 0.531220
[[0.53122]]
Generation 1, Trees 2, Best cost: 0.00794521, Est: 0.39726073, h: 0.531217
[[0.5312167]]
0.38000000000000006
Generation 0, Trees 2, Best cost: 0.00753248, Est: 0.37662393, h: 0.531602
[[0.53160197]]
Generation 1, Trees 2, Best cost: 0.00753248, Est: 0.37662381, h: 0.531596
[[0.5315957]]
0.4
Generation 0, Tr

In [4]:
res_pop = []
for r in res:
    pop = copy.deepcopy(r.populations[0])
    pop.select_ids([np.argmin(pop.fitness)])
    res_pop.append(pop.configuration)
pop = res_pop[0]
for r in res_pop[1:]:
    pop.merge(r)
kgs.dill_save(kgs.code_dir + '../res/packings.pickle', pop)